In [6]:
import moviepy.editor as mp
import whisper
import os

def extract_audio_with_moviepy(video_path, audio_path):
    video = mp.VideoFileClip(video_path)
    video.audio.write_audiofile(audio_path)

def split_audio_by_interval(audio_path, interval=30):
    """Tách audio thành các đoạn nhỏ, mỗi đoạn dài `interval` giây."""
    audio = mp.AudioFileClip(audio_path)
    duration = audio.duration  # Tổng độ dài audio
    clips = []
    
    # Tạo các đoạn audio 30 giây
    for start_time in range(0, int(duration), interval):
        end_time = min(start_time + interval, duration)  # Đảm bảo không vượt quá độ dài audio
        clip = audio.subclip(start_time, end_time)
        clip_path = f"temp_clip_{start_time}.wav"
        clip.write_audiofile(clip_path, codec='pcm_s16le')  # Lưu thành file tạm
        clips.append(clip_path)
    
    return clips

def video_to_text_by_interval(video_path, output_txt_path, model_size="base", interval=30):
    audio_path = 'output_audio.wav'
    
    # Trích xuất audio từ video
    try:
        extract_audio_with_moviepy(video_path, audio_path)
    except Exception as e:
        print(f"Error extracting audio: {e}")
        return

    # Tách audio thành các đoạn nhỏ
    audio_clips = split_audio_by_interval(audio_path, interval=interval)
    
    # Tải mô hình Whisper lớn
    model = whisper.load_model(model_size)
    
    all_text = []  # Chứa toàn bộ văn bản từ tất cả các đoạn
    
    # Chuyển đổi từng đoạn audio thành text
    for clip_path in audio_clips:
        try:
            result = model.transcribe(clip_path, language="vi")  # Thiết lập ngôn ngữ là tiếng Việt
            text = result["text"]
            all_text.append(f"[{text.strip()}]")  # Lưu text với dấu []
        except Exception as e:
            print(f"Error transcribing {clip_path}: {e}")
    
    # Lưu tất cả text vào file
    with open(output_txt_path, 'w', encoding='utf-8') as f:
        f.write(" ".join(all_text))
    
    # Xóa các file audio tạm
    if os.path.exists(audio_path):
        os.remove(audio_path)
    for clip_path in audio_clips:
        if os.path.exists(clip_path):
            os.remove(clip_path)

    return all_text

# Sử dụng
video_path = "../../data/batch1/video/L01_V002.mp4"
output_txt_path = "L01_V001_transcription.txt"
transcribed_text = video_to_text_by_interval(video_path, output_txt_path)

MoviePy - Writing audio in output_audio.wav


MoviePy - Done.
MoviePy - Writing audio in temp_clip_0.wav


MoviePy - Done.
MoviePy - Writing audio in temp_clip_30.wav


MoviePy - Done.
MoviePy - Writing audio in temp_clip_60.wav


MoviePy - Done.
MoviePy - Writing audio in temp_clip_90.wav


...

MoviePy - Done.
MoviePy - Writing audio in temp_clip_930.wav


MoviePy - Done.
MoviePy - Writing audio in temp_clip_960.wav


MoviePy - Done.

100%|█████████████████████████████████████| 1.42G/1.42G [12:08<00:00, 2.10MiB/s]


KeyboardInterrupt: 

In [7]:
import re
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

text = """
[Đoạn thứ nhất chứa nhiều thông tin về kinh tế và chính trị. Đây là phần quan trọng liên quan đến sự phát triển kinh tế xã hội của các nước trong khu vực.]
[Đoạn thứ hai là về môi trường. Nó bao gồm thông tin về biến đổi khí hậu, các chính sách bảo vệ môi trường và những thách thức về tài nguyên thiên nhiên.]
[Đoạn thứ ba nói về công nghệ. Trong đó có những phát minh gần đây, các xu hướng công nghệ và tầm ảnh hưởng của chúng đối với cuộc sống con người.]
"""

paragraphs = re.findall(r'\[(.*?)\]', text)

summaries = []

for paragraph in paragraphs:
    summary = summarizer(paragraph, max_length=150, min_length=40, do_sample=False)
    summaries.append(summary[0]['summary_text'])

with open("result.txt", "w", encoding="utf-8") as file:
    for i, summary in enumerate(summaries):
        file.write(f"Tóm tắt đoạn {i+1}: {summary}\n\n")


Your max_length is set to 150, but your input_length is only 133. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)
Your max_length is set to 150, but your input_length is only 132. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)
Your max_length is set to 150, but your input_length is only 135. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=67)
